##### This notebook was set up by Erica Sturm. Send your questions and comments to esturm@bnl.gov.

## Import relevant libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
complete_path = os.getcwd()
if 'nb' in complete_path:
    os.chdir("..")

In [3]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib
import os
import pandas as pd
import yaml
from IPython.display import clear_output
import warnings

In [4]:
from sklearn.kernel_ridge import KernelRidge
import time
from mlnrg.utils.logger import log

In [5]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
from mlnrg.utils.mpl_at_utils import MPLAdjutant
from mlnrg.loader import NRGData

## Choose data sets

In [6]:
# Choose Anderson versus Kondo

#kondo = pd.read_csv('data/Kondo_411267_trials_with_headers.csv')
anderson = pd.read_csv('data/Anderson_599578_trials_with_headers.csv')
#data_set = pd.read_csv('data/Kondo_411267_trials_with_headers.csv')
#data_set = pd.read_csv('/Users/erica/Documents/Research/2channel_anderson/First_17k_2CHAM_trials_with_headers.csv')

# For plotting
#grid = 'pd.read_csv(data/Omega_grid_values.csv')

#### Choose either furthest point sampling (FPS) or random point sampling (RPS) to train and test with. It is important to use these (without shfffling) so that the trials in the training/validation/test sets are the same as those in the NN!

In [9]:
#fps_indexes = pickle.load(open("results/fps_files/kondo_fps.pkl", "rb"))
#rps_indexes = pickle.load(open("results/fps_files/anderson_random.pkl", "rb"))
point_indexes = pickle.load(open("results/fps_files/anderson_fps.pkl", "rb"))

#### Specify number of trials to be used in training. Default is 50K!

In [15]:
total_used_trials = 49408 # Default = 50000
#all_training_idx = list(point_indexes[0]['train'][:total_used_trials])
all_training_idx = np.array(point_indexes[0]['train'][:])
all_validation_idx = list(point_indexes[0]['valid'][:])

In [17]:
from numpy import savetxt
savetxt("All_Anderson_training_data.csv", all_training_idx[:], delimiter=",", fmt='%d')

In [20]:
foo = (anderson.loc[all_training_idx[:], :]).to_numpy()
savetxt("All_Anderson_training_data.csv",foo[:], delimiter=",", fmt='%1.8f')

#### Symlog scaling here; comment or uncomment as necessary. 

In [ ]:
# # Start with B: rescale with symlog after adjusting for B=0 cases.
# m = (data_set[data_set['B'] != 0]['B']).abs().min()
# data_set.loc[list((data_set[data_set['B'] == 0]).index), 'B'] = m/10

# sign_B = np.sign(data_set['B'])
# data_set.loc[:, 'B'] = sign_B * np.log10(np.abs(data_set.loc[:, 'B']))
# #data_set.rename(columns={'B': 'symlog10 B'}, inplace=True)


# # Now do T: rescale with regular log after adjusting for T=0 cases.
# m = (data_set[data_set['T'] != 0]['T']).abs().min()
# data_set.loc[list((data_set[data_set['T'] == 0]).index), 'T'] = m/10
# data_set.loc[:, 'T'] = np.log10(data_set.loc[:, 'T'])
# #data_set.rename(columns={'T': 'log10 T'}, inplace=True)

# # Now do Gamma: recale with regular log.
# data_set.loc[:, 'Gamma'] = np.log10(data_set.loc[:, 'Gamma'])
# #data_set.rename(columns={'Gamma': 'log10 Gamma'}, inplace=True)

In [ ]:
# unscaled_tr_features = (data_set.loc[all_training_idx,'U':'T']).to_numpy()
# tr_targets = (data_set.iloc[all_training_idx, 8:]).to_numpy()

# unscaled_val_features = (data_set.loc[all_validation_idx,'U':'T']).to_numpy()
# val_targets = (data_set.iloc[all_validation_idx, 8:]).to_numpy()

unscaled_tr_features = (data_set.iloc[:16491,1:9]).to_numpy()
tr_targets = (data_set.iloc[:16491, 15:]).to_numpy()

unscaled_val_features = (data_set.iloc[16491:,1:9]).to_numpy()
val_targets = (data_set.iloc[16491:, 15:]).to_numpy()

#### Feature scaling

In [ ]:
mu_scaling = unscaled_tr_features.mean(axis=0)
std_scaling = unscaled_tr_features.std(axis=0)

tr_features = (unscaled_tr_features - mu_scaling) / std_scaling
val_features = (unscaled_val_features - mu_scaling) / std_scaling


unscaled_val_mean = np.mean(unscaled_val_features.mean(axis=0))
unscaled_val_std = np.mean(unscaled_val_features.std(axis=0))

rescaled_tr_mean = np.mean(tr_features.mean(axis=0))
rescaled_tr_std = np.mean(tr_features.std(axis=0))
rescaled_val_mean = np.mean(val_features.mean(axis=0))
rescaled_val_std = np.mean(val_features.std(axis=0))

log.info("\nThe original mean +/- std of the training features: %.4f +/- %.4f.\n" \
         "The rescaled mean +/- std of the training features: %.4f +/- %.4f.\n\n" \
         "The original mean +/- std of the validation features: %.4f +/- %.4f.\n" \
         "The rescaled mean +/- std of the validation features: %.4f +/- %.4f."
         % (np.mean(mu_scaling), np.mean(std_scaling), 
           rescaled_tr_mean, rescaled_tr_std,
           unscaled_val_mean, unscaled_val_std,
           rescaled_val_mean, rescaled_val_std
           )
        )

In [ ]:
print(unscaled_tr_features.mean(axis=0))
print(unscaled_tr_features.std(axis=0))
print(unscaled_val_features.mean(axis=0))
print(unscaled_val_features.std(axis=0))

In [ ]:
print(tr_features.mean(axis=0))
print(val_features.mean(axis=0))

## Set up model, train, and run validation set

#### Set up the KRR model: choose values for "gamma" the kernel radius and "alpha" the regularization term. The kernel is a constant set to "Laplacian": $k(x,y) = \mathrm{exp}(-\gamma||x-y||_1)$. The defaults are gamma=0.1 and alpha=0 (the same as Arsenault's values). Note that in the literature the kernel radius is denoted $\frac{1}{\sigma}$ and the regularization term is $\lambda$.

In [ ]:
alpha = 0.01  # Default = 0.0; other choices: [0.0, 0.01, 0.1]
gamma = 0.1 # Default = 0.1; other choices: [0.01, 0.1, 1]

# Set up KRR model
krrModel = KernelRidge(kernel='laplacian', gamma=gamma, alpha=alpha)
t0 = time.time()
# Fit the training data.
trained_model = krrModel.fit(tr_features, tr_targets)
fit_time = time.time() - t0
log.info("\nKRR model with Laplacian radius %.5g and regularization %.5g took %3.2g seconds to train on %d trials." 
      % (gamma, alpha, fit_time, len(tr_features)))

#### Make predictions abou the validation set using the trained KRR model and compute errors.

In [ ]:
t0 = time.time()
val_predictions = trained_model.predict(val_features[:,:])
prediction_time = time.time() - t0
log.info("\nThe prediction time for %d validation trials took %.2g seconds." %(len(val_features), prediction_time))

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    tr_cost = trained_model.score(tr_features, tr_targets)
    val_cost = trained_model.score(val_features, val_targets)
    log.info("\nThe cost of the training data (should be 1.0):\t%.5g\nThe cost of the validation data:\t\t%.5g"
     %(tr_cost, val_cost)
     )

In [ ]:
errors = np.mean(np.abs(val_targets - val_predictions), axis=1)
sorted_errors = []
for ii in range(len(errors)):
  sorted_errors.append([ii, val_targets[ii], val_predictions[ii], errors[ii]])

sorted_errors.sort(key=lambda x: x[3])

## Plot Results

In [ ]:
plt.clf()

y_Height = 0.05 * len(errors)
mean = np.log10(np.mean(errors))
median = np.log10(np.median(errors))

plt.hist(np.log10(errors), bins=60, color='k')
for ii in range(1, 10):
    x = np.log10(np.percentile(errors, ii*10))
    plt.plot((x, x), (0, y_Height), color='r', label="Deciles" if ii == 1 else None)
plt.plot((mean, mean), (0, y_Height), color='orange', label=f"Mean: {mean:.05g}")
plt.plot((median, median), (0, y_Height), color='b', label=f"Median: {median:.05g}")

plt.title("%d Anderson Trials KRR MAE" % len(errors))
plt.xlabel("$log_{10}(\mathrm{MAE})$")
plt.ylabel("Occurances")
plt.legend(loc="upper left", frameon=False)

#plt.savefig("Hist_anderson_fps_a" + str(alpha) + "_g" + str(gamma) +"_errors.png", dpi=300, bbox_inches='tight')
plt.show()

## Save relevant information.

In [ ]:
krr_info = {
    "set": "kondo",
    "sampling": "fps",
    "tr_set_size": len(tr_features),
    "val_set_size": len(val_features),
    "trained_model": trained_model,
    "fit_time": fit_time,
    "sorted_errors": sorted_errors,
    "tr_cost": tr_cost,
    "val_cost": val_cost
}

### MODIFY THIS CELL TO ENSURE YOU DO NOT OVERWRITE A PREVIOUS SAVE. 

In [ ]:
pickle.dump(krr_info, open("results/DC-KRR/KRR_kondo_fps_alpha0_gamma1_subset1_comparison.pkl", "wb" ))

In [ ]:
mean = np.mean(errors)
median = np.median(errors)
std = np.std(errors)
log.info("\nThe validation set's median: %.5f\nand mean +/- std: %.5f +/- %.5f" \
         "\n\nThe same, but log10: median: %.5f\nmean +/- std: %.5f +/- %.5f"
        % (median, mean, std, np.log10(median), np.log10(mean), np.log10(std))
        )



In [ ]:
alpha

In [ ]:
gamma
